In [1]:
!pip install pyodbc

In [2]:
!pip install selenium

In [3]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
import selenium
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import *

%autosave 120

【現在時間】2022/12/20 02:19:49
【所在目錄】C:\Users\kunia\py390
【Python】3.9.12
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [4]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

In [5]:
def investor(stockcode):
    url=f"https://goodinfo.tw/tw/ShowBuySaleChart.asp?STOCK_ID={stockcode}&CHT_CAT=MONTH"   #法人持股:一年

    chrome=webdriver.Chrome(executable_path='chromedriver')

    chrome.get(url)

    chrome.find_element_by_css_selector("#divBuySaleDetailData > table > tbody > tr > td > table > tbody > tr > td:nth-child(1) > nobr:nth-child(1) > select > option:nth-child(2)").click()
    time.sleep(5)
    chrome.find_element_by_css_selector("#divBuySaleDetailData > table > tbody > tr > td > table > tbody > tr > td:nth-child(2) > nobr:nth-child(1) > select > option:nth-child(3)").click()
    time.sleep(5)
    result=pd.read_html(chrome.page_source)

    data=result[25]
    chrome.quit()
    
    return data
def kchart(stockcode):
    url=f"https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={stockcode}&CHT_CAT2=MONTH"   #k線圖:一年

    chrome=webdriver.Chrome(executable_path='chromedriver')

    chrome.get(url)
    time.sleep(2)
    #chrome.find_element_by_css_selector("#sellK_ChartPeriod > option:nth-child(3)").click()
    time.sleep(2)
    result=pd.read_html(chrome.page_source)

    data=result[25]
    chrome.quit()
    
    return data

def shareholder(stockcode):
    url=f"https://goodinfo.tw/tw/EquityDistributionCatHis.asp?STOCK_ID={stockcode}"   #股東結構:比例

    chrome=webdriver.Chrome(executable_path='chromedriver')

    chrome.get(url)

    result=pd.read_html(chrome.page_source)

    data=result[17]
    chrome.quit()
    
    return data

In [17]:
stock_id = "1605"  #1605 1218 6446
table_s=kchart(stock_id)
df = table_s.dropna(axis='columns')
df

交易月份 交易日數     開盤     最高     最低     收盤      漲跌  漲跌(%) 振幅(%)  成交張數  ...  \
      交易月份 交易日數     開盤     最高     最低     收盤      漲跌  漲跌(%) 振幅(%)    千張  ...   
0    22M12   13     48  49.85     46  46.95   +0.25  +0.54  8.24  1107  ...   
1    22M11   22  36.05   49.3   35.4   46.7  +10.95  +30.6  38.9  2179  ...   
2    22M10   20   39.8   41.5   33.9  35.75   -4.75  -11.7  18.8   812  ...   
3    22M09   21   39.2   43.2   37.3   40.5    +0.8  +2.02  14.9  1224  ...   
4    22M08   23   34.1   40.9  31.15   39.7    +5.8  +17.1  28.8  1385  ...   
..     ...  ...    ...    ...    ...    ...     ...    ...   ...   ...  ...   
128  13M04   20    8.7   9.28   8.24   9.15   +0.52  +6.03  12.1   278  ...   
129  13M03   21   10.3  10.45   8.49   8.63   -1.57  -15.4  19.2   197  ...   
130  13M02   13   9.95   10.5   9.82   10.2   +0.25  +2.51  6.83   109  ...   
131  13M01   22   10.1   10.5   9.55   9.95   +0.03   +0.3  9.58   158  ...   
132  12M12   21   9.16   10.3   8.88   9.92    +0.8  +8.77  15.6   208  ...   

    法人買賣超(千張)                      外資持股(%) 融資(千張)       融券(千張)       券資比(%)  
           外資     投信     自營     合計 外資持股(%)     增減    餘額     增減    餘額 券資比(%)  
0       +37.1    -10  -1.91  +25.2    25.9  +3.39  52.4  +1.83    11     21  
1        +166  -74.4  +0.27  +92.2    25.4  +2.15    49  +1.33  9.19   18.8  
2       -49.5   +7.8  -3.74  -45.4    20.8  -1.42  46.8  -4.02  7.86   16.8  
3       -39.6  +36.7  -9.86  -12.8      22  +2.64  48.3  +1.98  11.9   24.6  
4       -71.1  +35.6  +9.62  -25.9    22.9  +7.69  45.6  +3.22   9.9   21.7  
..        ...    ...    ...    ...     ...    ...   ...    ...   ...    ...  
128     -42.9  +0.38  +0.04  -42.5    29.9  +6.73  89.8  +0.54  1.13   1.26  
129     -21.1  -2.45   -1.3  -24.8      31   +6.7  83.1  -2.25  0.59   0.71  
130     +26.2      0  +2.08  +28.2    31.6  -3.93  76.4  +0.18  2.84   3.72  
131     +21.5  +1.36  +0.33  +23.2    30.9  -5.67  80.3   -0.3  2.66   3.31  
132     +53.4  -0.14   +1.4  +54.7    30.4  -4.84    86  +1.15  2.95   3.43  

[133 rows x 23 columns]

In [18]:
products_list = df.values.tolist()
print(len(products_list))

del products_list[119]
del products_list[118]
del products_list[99]
del products_list[98]
del products_list[79]
del products_list[78]
del products_list[59]
del products_list[58]
del products_list[39]
del products_list[38]
del products_list[19]
del products_list[18]
lists_num=len(products_list)
print(len(products_list))
print(products_list)

133
121
[['22M12', '13', '48', '49.85', '46', '46.95', '+0.25', '+0.54', '8.24', '1107', '85.2', '531', '40.8', '+37.1', '-10', '-1.91', '+25.2', '25.9', '+3.39', '52.4', '+1.83', '11', '21'], ['22M11', '22', '36.05', '49.3', '35.4', '46.7', '+10.95', '+30.6', '38.9', '2179', '99', '951', '43.2', '+166', '-74.4', '+0.27', '+92.2', '25.4', '+2.15', '49', '+1.33', '9.19', '18.8'], ['22M10', '20', '39.8', '41.5', '33.9', '35.75', '-4.75', '-11.7', '18.8', '812', '40.6', '301', '15.1', '-49.5', '+7.8', '-3.74', '-45.4', '20.8', '-1.42', '46.8', '-4.02', '7.86', '16.8'], ['22M09', '21', '39.2', '43.2', '37.3', '40.5', '+0.8', '+2.02', '14.9', '1224', '58.3', '493', '23.5', '-39.6', '+36.7', '-9.86', '-12.8', '22', '+2.64', '48.3', '+1.98', '11.9', '24.6'], ['22M08', '23', '34.1', '40.9', '31.15', '39.7', '+5.8', '+17.1', '28.8', '1385', '60.2', '508', '22.1', '-71.1', '+35.6', '+9.62', '-25.9', '22.9', '+7.69', '45.6', '+3.22', '9.9', '21.7'], ['22M07', '21', '35.8', '36.1', '31.35', '33.9'

In [19]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")

In [20]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令


sqlStr ='''create table 當月走勢表
(
	當月走勢表編號 nvarchar(20) primary key
	,[開盤] nvarchar(20) Null
	,[當月最高] nvarchar(20) Null
    ,[當月最低] nvarchar(20) NULL
	,[當月收盤] nvarchar(20) Null
	,[開盤日期*] nvarchar(10) not Null
	,[股票代號*] nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

In [21]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令
for i in range(236,lists_num):
    Dstock_id='D'+str(stock_id)+"2021/"+str(products_list[i][0])
   
    sqlStr = f""" 
    Insert into 當月走勢表 (當月走勢表編號, 
    [開盤],
    [當月最高],[當月最低],
    [當月收盤],[開盤日期*],[股票代號*])
    values ('{str(Dstock_id)}',
    '{products_list[i][1]}','{products_list[i][2]}','{products_list[i][3]}','{products_list[i][4]}'
    ,'{str("2021/"+products_list[i][0])}','{stock_id}')
    
    """
    cursor.execute(sqlStr)

print(sqlStr)

create table 當月走勢表
(
	當月走勢表編號 nvarchar(20) primary key
	,[開盤] nvarchar(20) Null
	,[當月最高] nvarchar(20) Null
    ,[當月最低] nvarchar(20) NULL
	,[當月收盤] nvarchar(20) Null
	,[開盤日期*] nvarchar(10) not Null
	,[股票代號*] nvarchar(10) not Null
)

